# Updating FSC Dataframe to search specifically for matching dates instead of file numbers

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 28 15:42:34 2024

@author: s1803229
"""

from scripts.imports import os, glob, pdb, np, h5py, pd, xr, gpd, Proj, Transformer, CRS, \
                        plt, cmap, cmap2, Model, Data, ODR, datetime, rasterio, show, \
                        ccrs, cfeature
import seaborn as sns
from scripts.classes_fixed import *
from scripts.track_pairs import *
from scripts.show_tracks import *
from scripts.parallel import pvpg_parallel

# Function to compute mean without the warning
def safe_nanmean(slice):
    if len(slice) == 0 or np.isnan(slice).all():
        return np.nan
    else:
        return np.nanmean(slice)

def parse_filename_datetime(filename):
    # Extracting only the filename from the full path
    filename_only = filename.split('/')[-1]
    
    # Finding the index of the first appearance of 'ATL03_' or 'ATL08_'
    atl03_index = filename_only.find('ATL03_')
    atl08_index = filename_only.find('ATL08_')
    
    # Determining the split index based on which string appears first or if neither is found
    split_index = min(filter(lambda x: x >= 0, [atl03_index, atl08_index]))

    # Extracting yyyymmddhhmmss part
    date_str = filename_only[split_index + 6:split_index + 20]
    datetime_obj = datetime.strptime(date_str, '%Y%m%d%H%M%S')
    
    return datetime_obj

def datetime_to_date(datetime_obj):
    return datetime_obj.strftime('%d/%m/%Y')
    
def FSC_dataframe(dirpath, csv_path):
    all_ATL03, all_ATL08 = track_pairs(dirpath)
    N = len(all_ATL03)

    foldername = dirpath.split('/')[-2]
    
    df = pd.read_csv(csv_path).drop('Image', axis=1).dropna()
    
    pvpg = []
    mean_Eg_strong = []
    mean_Eg_weak = []
    mean_Ev_strong = []
    mean_Ev_weak = []
    msw_flags = []
    night_flags = []
    asrs = []
    
    for i, (atl03_filepath, atl08_filepath) in enumerate(zip(all_ATL03, all_ATL08)):
        filedate = datetime_to_date(parse_filename_datetime(atl03_filepath))
        if (filedate in df['Date'].values) and (#######################################################################
        coefs,means,msw_flag,night_flag,asr = pvpg_parallel(all_ATL03[int(i)], all_ATL08[int(i)], file_index = int(i),loss='arctan')
        pvpg.append(-coefs[0])
        mean_Eg_strong.append(safe_nanmean(means[0]))
        mean_Eg_weak.append(safe_nanmean(means[1]))
        mean_Ev_strong.append(safe_nanmean(means[2]))
        mean_Ev_weak.append(safe_nanmean(means[3]))
        msw_flags.append(msw_flag)
        night_flags.append(night_flag)
        asrs.append(asr)
    
    df['pvpg'] = pvpg
    df['mean_Eg_strong'] = mean_Eg_strong
    df['mean_Eg_weak'] = mean_Eg_weak
    df['mean_Ev_strong'] = mean_Ev_strong
    df['mean_Ev_weak'] = mean_Ev_weak
    df['msw_flag'] = msw_flags
    df['night_flag'] = night_flags
    df['asr'] = asrs
    
    #make categorical column
    df['Joint Snow'] = df['Joint Snow'].astype('category')
    df['FSC'] = df['FSC'].astype('category')
    df['Tree Snow'] = df['Tree Snow'].astype('category')
    
    df_pure = df.drop(['Camera','Date'], axis=1)
    
    return df, df_pure

In [11]:
%matplotlib ipympl

dirpath = '../data/sodankyla_full/'
csv_path = 'snow_cam_details.csv'

all_ATL03, all_ATL08 = track_pairs(dirpath)
N = len(all_ATL03)

df = pd.read_csv(csv_path).dropna().drop('Image', axis=1)

for i in range(3,5):
    A = datetime_to_date(parse_filename_datetime(all_ATL08[i]))
    print(A)
    if A in df['Date'].values:
        print(1)
        
    else:
        print(0)

# dirpath.split('/')[-2]

# df

26/12/2018
0
12/01/2019
1


In [7]:
for i, (x, y) in enumerate(zip([1,2,3],[4,5,6])):
    print(i, x, y)

0 1 4
1 2 5
2 3 6
